In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
from script import load_images_to_tensors, load_csv_to_tensors, calculate_TC_in_CA, Annotation, calculate_areas_scalar, transform_points, points_to_pixels
import torch
import json
from PIL import Image
import math

In [31]:
# Usage example:
image_folder_train = "./ocelot2023_v1.0.1/annotations/train/tissue"
image_folder_test = "./ocelot2023_v1.0.1/annotations/test/tissue"
image_folder_val = "./ocelot2023_v1.0.1/annotations/val/tissue"
tissue_tensors = load_images_to_tensors(image_folder_train)
tissue_tensors += load_images_to_tensors(image_folder_test)
tissue_tensors += load_images_to_tensors(image_folder_val)

directory_path_train = "ocelot2023_v1.0.1/annotations/train/cell"
directory_path_test = "ocelot2023_v1.0.1/annotations/test/cell"
directory_path_val = "ocelot2023_v1.0.1/annotations/val/cell"

cell_tensors = load_csv_to_tensors(directory_path_train)
cell_tensors += load_csv_to_tensors(directory_path_test)
cell_tensors += load_csv_to_tensors(directory_path_val)

## Visualizatoin of image tissue 288

In [142]:
cell_area, tissue_area, scalar = calculate_areas_scalar("288")

In [155]:
cell_points = cell_tensors[287]
tissue = tissue_tensors[287]
cell_annotation = cell_points[:, 2]
cell_points = cell_points[:, :2]

print(cell_points[0, :])
cell_points = points_to_pixels(cell_points, cell_area, tissue_area, scalar)
cell_points = cell_points.to(dtype=torch.int)
print(cell_points[0, :])

tensor([741, 526])
tensor([697, 643], dtype=torch.int32)


In [144]:
print(tissue.size())
tissue_rgb = tissue.repeat(3, 1, 1)
print(tissue_rgb.size())

torch.Size([1, 1024, 1024])
torch.Size([3, 1024, 1024])


In [152]:
def draw_circle(tensor, center, radius, color):
    cx, cy = center

    for y in range(cy - radius, cy+radius):
        for x in range(cx-radius, cx+radius):
            distance = math.sqrt((x - cx)**2 + (y - cy)**2)
            if distance <= radius:
                tensor[:, y, x] = torch.tensor(color, dtype=torch.uint8)


# Drawing red circles around the points
radius = 3  # Radius of the circle
color_blue = [0, 0, 255]  # Blue color
color_yellow = [255, 255, 0]  # Yellow color
for point, annotation in zip(cell_points, cell_annotation):
    if annotation == 2:
        draw_circle(tissue_rgb, point, radius, color_blue)
    else:
        draw_circle(tissue_rgb, point, radius, color_yellow)

In [157]:
img = Image.fromarray(tissue_rgb.permute(1, 2, 0).numpy(), mode="RGB")
img.show() 

## Calculate tumor cells in cancer area

In [165]:
table = calculate_TC_in_CA(cell_tensors, tissue_tensors)

In [166]:
print(tissue_tensors[0].size())

print(table)
print(
    f"Count of cells with annotation 2 and point value 2: {table.get(Annotation((2, 2)).name)/(table.get(Annotation((1, 2)).name) + table.get(Annotation((2, 2)).name))}"
)

print(sum(table.values()))

torch.Size([1, 1024, 1024])
{'TC_CA': 57976, 'TC_non_CA': 11650, 'BC_non_CA': 32113, 'C1_UNK': 1507, 'C2_UNK': 2065, 'BC_CA': 7715, 'Cneg1_1': 5, 'Cneg1_UNK': 1, 'Cneg1_2': 1}
Count of cells with annotation 2 and point value 2: 0.8825562101353306
113033
